In [1]:
import src
import torch

In [2]:
# Load checkpoint and data

dm = src.datamodules.SSTDataModule(32, "roberta-large")
model = src.models.SSTModel.load_from_checkpoint("../logs/runs/2022-01-02/14-46-39/checkpoints/epoch_000.ckpt")

No config specified, defaulting to: sst/default
Reusing dataset sst (/home/studio-lab-user/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff)


  0%|          | 0/3 [00:00<?, ?it/s]

Loading cached processed dataset at /home/studio-lab-user/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-43f5bb448e8129c6.arrow
Loading cached processed dataset at /home/studio-lab-user/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-eead062aa69beb34.arrow
Loading cached processed dataset at /home/studio-lab-user/.cache/huggingface/datasets/sst/default/1.0.0/b8a7889ef01c5d3ae8c379b84cc4080f8aad3ac2bc538701cbe0ac6416fb76ff/cache-0ef065857ea93057.arrow
Some weights of the model checkpoint at roberta-large were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a 

In [9]:
# Run prediction and extract the labels and predictions of examples where the binary classification is wrong

import tqdm

model.eval()
model.cuda()

data = []

with torch.no_grad():
    # for batch in tqdm.tqdm(dm.test_dataloader()):
    for batch in tqdm.tqdm(dm.val_dataloader()):

        for field in batch:
            batch[field] = batch[field].cuda()

        out = model(batch)
        mask = torch.where(out > .5, 1, 0) != torch.where(batch["label"] > .5, 1, 0)
        data.extend([(pred.item(), label.item()) for pred, label in zip(out[mask], batch["label"][mask])])

100%|██████████| 70/70 [00:24<00:00,  2.82it/s]


In [14]:
# Show Histogram of labels of the wrong examples
# --> model is mostly just wrong on examples close to 0.5

torch.tensor([x[1] for x in data]).histogram(10, range=(0,1))

torch.return_types.histogram(
hist=tensor([ 1.,  4., 20., 42., 62., 72., 13.,  2.,  3.,  1.]),
bin_edges=tensor([0.0000, 0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000, 0.7000, 0.8000,
        0.9000, 1.0000]))

In [5]:
train_labels = dm.dataset_dict["train"]["label"]

In [9]:
(train_labels == 0).sum()

tensor(16)

In [10]:
model = src.models.SSTModel.load_from_checkpoint("../logs/runs/2022-01-07/08-36-52/checkpoints/epoch_000.ckpt")

Some weights of the model checkpoint at distilroberta-base were not used when initializing RobertaModel: ['lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.bias', 'lm_head.decoder.weight', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias']
- This IS expected if you are initializing RobertaModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
# Run prediction and extract the labels and predictions of examples where the binary classification is wrong

import tqdm

model.eval()
model.cuda()

data = []

with torch.no_grad():
    # for batch in tqdm.tqdm(dm.test_dataloader()):
    for batch in tqdm.tqdm(dm.val_dataloader()):

        for field in batch:
            batch[field] = batch[field].cuda()

        out = model(batch)
        mask = torch.where(out > .5, 1, 0) != torch.where(batch["label"] > .5, 1, 0)
        data.extend([(pred.item(), label.item()) for pred, label in zip(out[mask], batch["label"][mask])])
        
torch.tensor([x[1] for x in data]).histogram(10, range=(0,1))

100%|██████████| 35/35 [00:02<00:00, 14.78it/s]


torch.return_types.histogram(
hist=tensor([ 0.,  3., 10., 24., 33., 50., 24., 25.,  6.,  2.]),
bin_edges=tensor([0.0000, 0.1000, 0.2000, 0.3000, 0.4000, 0.5000, 0.6000, 0.7000, 0.8000,
        0.9000, 1.0000]))